In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, align_atlas, DK52_centers

Connecting dklab@192.168.1.12:3306


In [2]:
animal = 'MD589'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')

In [3]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.452
width: 43700
height: 32400
Shape of atlas volume (1464, 1975, 447)


In [4]:
reference_centers = sql_controller.get_centers_dict(animal)

In [5]:
reference_centers

{'10N_L': [31002.1, 17139.3, 210.0],
 '10N_R': [30851.8, 17026.3, 242.0],
 '4N_L': [25238.4, 13606.0, 210.0],
 '4N_R': [25231.8, 13572.2, 236.0],
 '5N_L': [25863.9, 16448.5, 160.0],
 '5N_R': [25617.9, 16089.0, 298.0],
 '7N_L': [27315.2, 18976.5, 174.0],
 '7N_R': [27227.1, 18547.7, 296.0],
 '7n_L': [26920.5, 16996.3, 177.0],
 '7n_R': [26803.3, 16688.2, 284.0],
 'Amb_L': [29043.0, 18890.2, 167.0],
 'Amb_R': [28901.5, 18291.1, 296.0],
 'DC_L': [28764.5, 15560.1, 134.0],
 'DC_R': [28519.2, 14960.1, 330.0],
 'LC_L': [26993.8, 15147.0, 180.0],
 'LC_R': [26951.6, 14929.4, 268.0],
 'Pn_L': [23019.2, 17948.5, 200.0],
 'Pn_R': [23067.2, 17945.9, 270.0],
 'SC': [24976.4, 10136.9, 220.0],
 'Tz_L': [25210.3, 18857.2, 212.0],
 'Tz_R': [25142.5, 18757.5, 262.0]}

In [6]:
%%time
reference_centers = sql_controller.get_centers_dict('DK52')
atlas_box_size=(2010,1490,  447)
align_atlas(reference_centers)
#sql_controller.get_centers_dict('Atlas')

No histology for Atlas
No scan run for Atlas
Please type the following numbers into neuroglancer to align the atlas structures.

Rotation:
[[ 0.99539937  0.36001946  0.01398442]
 [-0.35951647  0.99520383 -0.03076871]
 [-0.02361112  0.02418248  1.05805822]]

Translation:
[[19186.25959077]
 [ 9825.29346238]
 [   78.18300782]]
CPU times: user 122 ms, sys: 173 µs, total: 122 ms
Wall time: 186 ms


(array([[ 0.99539937,  0.36001946,  0.01398442],
        [-0.35951647,  0.99520383, -0.03076871],
        [-0.02361112,  0.02418248,  1.05805822]]),
 array([[19186.25959077],
        [ 9825.29346238],
        [   78.18300782]]))

In [10]:
def get_db_structure_infos():
    sqlController = SqlController('MD589')
    db_structures = sqlController.get_structures_dict()
    return db_structures

def get_known_foundation_structure_names():
    known_foundation_structures = ['MVePC', 'DTgP', 'VTA', 'Li', 'Op', 'Sp5C', 'RPC', 'MVeMC', 'APT', 'IPR',
                                   'Cb', 'pc', 'Amb', 'SolIM', 'Pr5VL', 'IPC', '8n', 'MPB', 'Pr5', 'SNR',
                                   'DRD', 'PBG', '10N', 'VTg', 'R', 'IF', 'RR', 'LDTg', '5TT', 'Bar',
                                   'Tz', 'IO', 'Cu', 'SuVe', '12N', '6N', 'PTg', 'Sp5I', 'SNC', 'MnR',
                                   'RtTg', 'Gr', 'ECu', 'DTgC', '4N', 'IPA', '3N', '7N', 'LC', '7n',
                                   'SC', 'LPB', 'EW', 'Pr5DM', 'VCA', '5N', 'Dk', 'DTg', 'LVe', 'SpVe',
                                   'MVe', 'LSO', 'InC', 'IC', 'Sp5O', 'DC', 'Pn', 'LRt', 'RMC', 'PF',
                                   'VCP', 'CnF', 'Sol', 'IPL', 'X', 'AP', 'MiTg', 'DRI', 'RPF', 'VLL']
    return known_foundation_structures

In [12]:
db_structure_infos = get_db_structure_infos()
known_foundation_structure_names = get_known_foundation_structure_names()
non_db_structure_names = [structure for structure in known_foundation_structure_names if structure not in db_structure_infos.keys()]


In [17]:
[(number, f'{structure}: {label}') for structure, (label, number) in db_structure_infos.items()]

[(1, '10N_L: Dorsal nucleus of vagus nerve'),
 (1, '10N_R: Dorsal nucleus of vagus nerve'),
 (2, '12N: Hypoglossal nucleus'),
 (3, '3N_L: Oculomotor nucleus'),
 (3, '3N_R: Oculomotor nucleus'),
 (4, '4N_L: Trochlear nucleus'),
 (4, '4N_R: Trochlear nucleus'),
 (5, '5N_L: Trigeminal motor nucleus'),
 (5, '5N_R: Trigeminal motor nucleus'),
 (6, '6N_L: Abducens nucleus'),
 (6, '6N_R: Abducens nucleus'),
 (7, '7N_L: Facial nucleus'),
 (7, '7N_R: Facial nucleus'),
 (9, '7n_L: Facial nerve'),
 (9, '7n_R: Facial nerve'),
 (28, 'AP: Area postrema'),
 (8, 'Amb_L: Nucleus ambiggus'),
 (8, 'Amb_R: Nucleus ambiggus'),
 (10, 'DC_L: Dorsal cochlea nucleus'),
 (10, 'DC_R: Dorsal cochlea nucleus'),
 (11, 'IC: Inferior colliculus'),
 (12, 'LC_L: Locus corelus'),
 (12, 'LC_R: Locus corelus'),
 (13, 'LRt_L: Lateral reticular nucleus'),
 (13, 'LRt_R: Lateral reticular nucleus'),
 (14, 'PBG_L: Parabigeminal nucleus'),
 (14, 'PBG_R: Parabigeminal nucleus'),
 (15, 'Pn_L: Pontine grey'),
 (15, 'Pn_R: Pontine 

In [ ]:
segment_properties = [(number, f'{structure}: {label}') for structure, (label, number, sided) in db_structure_infos.items()]
if all_known:
    segment_properties += [(len(db_structure_infos) + index + 1, structure) for index, structure in enumerate(non_db_structure_names)]
